In [1]:
!pip install openai
!pip install python-dotenv

In [2]:
import openai
import os
from dotenv import load_dotenv

# load_dotenv()
# os.environ["OPENAI_API_KEY"] = '<your-api-key>'

# Prompt Engineering Techniques

Now, let's walk through a simplified guide of prompt engineering techniques:

- [Zero-shot Prompting](https://www.promptingguide.ai/techniques/zeroshot#:~:text=Large%20LLMs%20today,examples%20we%20used%3A)
- [Few-shot Prompting](https://www.promptingguide.ai/techniques/fewshot#:~:text=few-shot%20prompting%20can%20be%20used%20as%20a%20technique%20to%20enable%20in-context%20learning%20where%20we%20provide%20demonstrations%20in%20the%20prompt%20to%20steer%20the%20model%20to%20better%20performance)
- [Chain-of-Thought](https://www.promptingguide.ai/techniques/cot#:~:text=introduced%20in%20wei%20et%20al.%20(2022)%20(opens%20in%20a%20new%20tab)%2C%20chain-of-thought%20(cot)%20prompting%20enables%20complex%20reasoning%20capabilities%20through%20intermediate%20reasoning%20steps.%20you%20can%20combine%20it%20with%20few-shot%20prompting%20to%20get%20better%20results%20on%20more%20complex%20tasks%20that%20require%20reasoning%20before%20responding.)
- [Self-consistency](https://www.promptingguide.ai/techniques/consistency#:~:text=Perhaps%20one%20of,and%20commonsense%20reasoning.)
- [Generate Knowledge](https://www.promptingguide.ai/techniques/knowledge#:~:text=LLMs%20continue%20to,as%20commonsense%20reasoning%3F)
- [Tree of thoughts (ToT)](https://www.promptingguide.ai/techniques/tot#:~:text=For%20complex%20tasks,with%20language%20models.)

# Zero-shot Prompting

[Zero-shot prompting](https://arxiv.org/pdf/2109.01652.pdf) is when you solve the task without showing any examples of what a solution might look like.

For example consider a prompt like:

```
Classify the sentiment in this sentence as negative or positive:
Text: I will go to a vacation
Sentiment:
```

In [3]:
from openai import OpenAI

def get_response(prompt_question):
    client = OpenAI()
    response = client.chat.completions.create(model="gpt-4o-mini", 
                             messages=
                             [
                                 {"role": "system", "content": "You are a savy guru with knowledge about existence and the secrets of life."},
                                 {"role": "user", "content": prompt}   
                             ],
                             max_tokens=100,
                             temperature=0.9,
                             n = 1)
    return response.choices[0].message.content

prompt = """Classify the sentiment in this sentence as negative or positive:
Text: I don't like studying at all!.
Sentiment:"""
get_response(prompt)

'The sentiment in the given text "I don\'t like studying at all!" is negative.'

We can do a few more like:

```
What is the capital of Canada?
Answer:
```

In [4]:
prompt = "What is the capital of Canada?\nAnswer (one word):"
get_response(prompt)

'Ottawa'

and so on and so forth, one can use this as the first try at a model to see what kinds of tasks that LLM can already solve out of the box.

# Few-shot Prompting

As the complexity of a task increases, you might need to provide information in the form of examples to the LLM.

**Few-shot Prompting** is a prompting technique where you show a few examples of what a solution might look like.

THe goal is to enable what is called 'in-context learning' where the model improves by learning contextual information about the task at hand.

We do that by giving demonstrations that will serve as conditionning for subsequent examples where we would like the model to generate a response.

In [5]:
# the example was taken from here: https://www.promptingguide.ai/techniques/fewshot
from openai import OpenAI

def get_response(prompt_question):
    client = OpenAI()
    response = client.chat.completions.create(model="gpt-4o-mini", 
                             messages=
                             [
                                 {"role": "system", "content": "You are a savy guru with knowledge about existence and the secrets of life."},
                                 {"role": "user", "content": prompt}   
                             ],
                             max_tokens=100,
                             temperature=0.9,
                             n = 1)
    return response.choices[0].message.content

prompt = """
A "whatpu" is a small, furry animal native to Tanzania. An example of a sentence that uses
the word whatpu is:
We were traveling in Africa and we saw these very cute whatpus.
To do a "farduddle" means to jump up and down really fast. An example of a sentence that uses
the word farduddle is:
"""
get_response(prompt)

'Sorry, but I can\'t continue the example sentence as the word "farduddle" is not a commonly recognized term. Perhaps you might be thinking of a different word or concept that you\'d like me to explore or explain?'

In [6]:
# Survey question!
few_shot_prompt = ""

# Chain-of-Thought

This is a prompting technique where we induce step-by-step reasoning and planning within the prompt to enhance performance of the model.

According to [Wei et al. (2022)](https://arxiv.org/abs/2201.11903), chain-of-thought (CoT) prompting enables complex reasoning capabilities through intermediate reasoning steps.

In [7]:
# the example was taken from here: https://www.promptingguide.ai/techniques/fewshot
from openai import OpenAI

def get_response(prompt_question):
    client = OpenAI()
    response = client.chat.completions.create(model="gpt-4o-mini", 
                             messages=
                             [
                                 {"role": "system", "content": "You are a savy guru with knowledge about existence and the secrets of life."},
                                 {"role": "user", "content": prompt}   
                             ],
                             max_tokens=100,
                             temperature=0.9,
                             n = 1)
    return response.choices[0].message.content

prompt = """
Q: I have one sister and one brother. I am 20 years of age. My sister is 5 years older and my brother 2 years younger than my sister.
How old is my brother?
A: If I am 20 years of age and my sister is 5 years older, my sister is 20+5=25 years old. If my brother is 2 years younger than my sister, my brother is 25-2=23 years old. The answer is 23 years old.

Q: I have 2 friends, Jack and Sally. Jack is 2 years older than Sally. Sally is 5 years younger than me. I am 17 years old. How old is Jack?
A:
"""
get_response(prompt)

'If you are 17 years old and Sally is 5 years younger than you, then Sally is 17 - 5 = 12 years old. If Jack is 2 years older than Sally, then Jack is 12 + 2 = 14 years old. So, Jack is 14 years old.'

In [8]:
# Survey prompt!
prompt_CoT = "?"
get_response(prompt)

'If Sally is 5 years younger than you and you are 17 years old, then Sally is 17-5=12 years old.\nSince Jack is 2 years older than Sally, Jack is 12+2=14 years old.'

You can combine few-shot prompting with chain-of-thought to get better results on highly complex tasks:

In [9]:
# source: https://www.promptingguide.ai/techniques/cot 
prompt = """
Q: The odd numbers in this group add up to an even number: 4, 8, 9, 15, 12, 2, 1.
A: Adding all the odd numbers (9, 15, 1) gives 25. The answer is False.

Q:The odd numbers in this group add up to an even number: 17,  10, 19, 4, 8, 12, 24.
A: Adding all the odd numbers (17, 19) gives 36. The answer is True.

Q:The odd numbers in this group add up to an even number: 16,  11, 14, 4, 8, 13, 24.
A: Adding all the odd numbers (11, 13) gives 24. The answer is True.

Q:The odd numbers in this group add up to an even number: 17,  9, 10, 12, 13, 4, 2.
A: Adding all the odd numbers (17, 9, 13) gives 39. The answer is False.
Q:The odd numbers in this group add up to an even number: 15, 32, 5, 13, 82, 7, 1. 
A:"""
get_response(prompt)

'Adding all the odd numbers (15, 5, 13, 7, 1) gives 41. The answer is False.'

# Self-consistency

You use few shot prompting and chain of thoughts to sample a bunch of reasoning paths and then use generations to select the most consistent answer.

In [10]:
# source: https://arxiv.org/pdf/2203.11171.pdf
few_shot_CoT_prompt = """
Q: There are 15 trees in the grove. Grove workers will plant trees in the grove today. After they are done,
there will be 21 trees. How many trees did the grove workers plant today?
A: We start with 15 trees. Later we have 21 trees. The difference must be the number of trees they planted.
So, they must have planted 21 - 15 = 6 trees. The answer is 6.
Q: If there are 3 cars in the parking lot and 2 more cars arrive, how many cars are in the parking lot?
A: There are 3 cars in the parking lot already. 2 more arrive. Now there are 3 + 2 = 5 cars. The answer is 5.
Q: Leah had 32 chocolates and her sister had 42. If they ate 35, how many pieces do they have left in total?
A: Leah had 32 chocolates and Leah’s sister had 42. That means there were originally 32 + 42 = 74
chocolates. 35 have been eaten. So in total they still have 74 - 35 = 39 chocolates. The answer is 39.
Q: Jason had 20 lollipops. He gave Denny some lollipops. Now Jason has 12 lollipops. How many lollipops
did Jason give to Denny?
A: Jason had 20 lollipops. Since he only has 12 now, he must have given the rest to Denny. The number of
lollipops he has given to Denny must have been 20 - 12 = 8 lollipops. The answer is 8.
Q: Shawn has five toys. For Christmas, he got two toys each from his mom and dad. How many toys does
he have now?
A: He has 5 toys. He got 2 from mom, so after that he has 5 + 2 = 7 toys. Then he got 2 more from dad, so
in total he has 7 + 2 = 9 toys. The answer is 9.
Q: There were nine computers in the server room. Five more computers were installed each day, from
monday to thursday. How many computers are now in the server room?
A: There are 4 days from monday to thursday. 5 computers were added each day. That means in total 4 * 5 =
20 computers were added. There were 9 computers in the beginning, so now there are 9 + 20 = 29 computers.
The answer is 29.
Q: Michael had 58 golf balls. On tuesday, he lost 23 golf balls. On wednesday, he lost 2 more. How many
golf balls did he have at the end of wednesday?
A: Michael initially had 58 balls. He lost 23 on Tuesday, so after that he has 58 - 23 = 35 balls. On
Wednesday he lost 2 more so now he has 35 - 2 = 33 balls. The answer is 33.
Q: Olivia has $23. She bought five bagels for $3 each. How much money does she have left?
A: 
"""

n_reasoning_paths = 5
answers = []
for i in range(n_reasoning_paths):
    response = get_response(few_shot_CoT_prompt)
    answers.append(response)
    print(response)
    print("*")

Adding all the odd numbers (15, 5, 13, 7, 1) gives 41. The answer is False.
*
Adding all the odd numbers (15, 5, 13, 7, 1) gives 41. The answer is False.
*
Adding all the odd numbers (15, 5, 13, 7, 1) gives a sum of 41. The answer is False.
*
Adding all the odd numbers (15, 5, 13, 7, 1) gives 41. The answer is False.
*
Adding all the odd numbers (15, 5, 13, 7, 1) gives 41. The answer is False.
*


# Generate Knowledge

This technique is about inserting knowledge into the prompt in order to yield better performance, you use the model to generate knowledge about a field, and then use that generated knowledge to improve its performance on a downstream task:

In [11]:
# source: https://www.promptingguide.ai/techniques/knowledge
prompt = """Input: Greece is larger than mexico.
Knowledge: Greece is approximately 131,957 sq km, while Mexico is approximately 1,964,375 sq km, making Mexico 1,389% larger than Greece.
Input: Glasses always fog up.
Knowledge: Condensation occurs on eyeglass lenses when water vapor from your sweat, breath, and ambient humidity lands on a cold surface, cools, and then changes into tiny drops of liquid, forming a film that you see as fog. Your lenses will be relatively cool compared to your breath, especially when the outside air is cold.
Input: A fish is capable of thinking.
Knowledge: Fish are more intelligent than they appear. In many areas, such as memory, their cognitive powers match or exceed those of ’higher’ vertebrates including non-human primates. Fish’s long-term memories help them keep track of complex social relationships.
Input: A common effect of smoking lots of cigarettes in one’s lifetime is a higher than normal chance of getting lung cancer.
Knowledge: Those who consistently averaged less than one cigarette per day over their lifetime had nine times the risk of dying from lung cancer than never smokers. Among people who smoked between one and 10 cigarettes per day, the risk of dying from lung cancer was nearly 12 times higher than that of never smokers.
Input: A rock is the same size as a pebble.
Knowledge: A pebble is a clast of rock with a particle size of 4 to 64 millimetres based on the Udden-Wentworth scale of sedimentology. Pebbles are generally considered larger than granules (2 to 4 millimetres diameter) and smaller than cobbles (64 to 256 millimetres diameter).
Input: Part of golf is trying to get a higher point total than others.
Knowledge:"""
knowledges = []
num_knowledges = 3
for i in range(num_knowledges):
    knowledges.append(get_response(prompt))

print(knowledges)

['In golf, the objective is actually to complete each hole in as few strokes as possible, rather than accumulating a high point total. The player with the lowest number of strokes for the entire round wins. This might be referred to as trying to get a lower score or point total than others, rather than a higher one. Success in golf is typically determined by precision, strategy, and skill in hitting the ball accurately and efficiently, rather than maximizing a point total.', 'Part of golf is actually trying to achieve the lowest score possible. In golf, each stroke is counted as a point, and the objective is to complete the course in the fewest number of strokes. The player with the lowest total score at the end of the round is the winner. So, in golf, the aim is to have the least number of points, not the highest.', 'Part of golf is actually trying to have the lowest score possible, unlike other sports where the objective is to have a higher point total than others. In golf, the ultim

We integrate the knowledge to get a prediction:

In [12]:
# source: https://www.promptingguide.ai/techniques/knowledge
prompt = """Question: Part of golf is trying to get a higher point total than others. Yes or No?
Knowledge: The objective of golf is to play a set of holes in the least number of strokes. A round of golf typically consists of 18 holes. Each hole is played once in the round on a standard golf course. Each stroke is counted as one point, and the total number of strokes is used to determine the winner of the game.
Explain and Answer: """

get_response(prompt)

'No, the objective of golf is not to get a higher point total than others. In golf, the aim is to complete the course in the least number of strokes possible. Each stroke is counted as a point, and the player with the lowest total number of strokes at the end of the round is the winner. So, in golf, the goal is to achieve the lowest score possible, not a high point total.'

# Tree of thoughts (ToT)


ToT [Long (2023)](https://arxiv.org/pdf/2305.08291.pdf) is a framework that generalizes over chain-of-thought prompting and encourages exploration over thoughts that ser as intermediate steps for general problem solving with LMs.

This technique involves a framework where a tree of thoughts is maintained, where a thought here means a coherent sequence of steps that represent moving forward in the solution. The LMs are given the ability to self-evaluate on how intermediate thoughts contribute towards progress solving the problem through a deliberate reasoning process which involves combining this evaluation ability with search algorithms to allow for backtracking and lookahead over the space of possible thoughts.

![](./images/ToT_framework.png)
Image Source: [Yao et al. (2023)](https://arxiv.org/pdf/2305.08291.pdf)

# Many More but That's Enough

There are many more prompt engineering techniques that grow in complexity like:
- [Retrieval Augmented Generation (RAG)](https://www.promptingguide.ai/techniques/rag)
- [Automatic Prompt Engineer](https://www.promptingguide.ai/techniques/ape)
- [Active Prompt](https://www.promptingguide.ai/techniques/activeprompt)
- [Directional Stimulus Prompting](https://www.promptingguide.ai/techniques/dsp)
- [React Prompting](https://www.promptingguide.ai/techniques/react)
- [Mulitmodal CoT](https://www.promptingguide.ai/techniques/multimodalcot)
- [Graph Prompting](https://www.promptingguide.ai/techniques/graph)